## In this notebook we will identify various analytics by route

In [9]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

In [10]:
flights = pd.read_csv('./data/raw/flight_edges.tsv', sep='\t', header=None).rename(columns={0:'Origin', 1:'Destination', 2:'Origin City', 3:'Destination City', 4:'Passengers', 5:'Seats', 6:'Flights', 7:'Distance', 8:'Fly Date', 9:'Origin Population', 10: 'Destination Population'})
print(flights.shape)
flights.head()

(3606803, 11)


,Origin,Destination,Origin City,Destination City,Passengers,Seats,Flights,Distance,Fly Date,Origin Population,Destination Population
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254.0,200810,122049,86219
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103.0,199011,284093,76034
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103.0,199012,284093,76034
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103.0,199010,284093,76034
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156.0,199002,147300,76034


In [11]:
# Create Datetime Column
monthly = flights['Fly Date'].map(lambda x: str(int(str(x)[0:4])) + 'M' + str(int(str(x)[4:6])))
from statsmodels.tsa.base.datetools import dates_from_str
monthly = dates_from_str(monthly)
flights['year-month']= pd.DatetimeIndex(monthly)
flights.head()

,Origin,Destination,Origin City,Destination City,Passengers,Seats,Flights,Distance,Fly Date,Origin Population,Destination Population,year-month
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254.0,200810,122049,86219,2008-10-31
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103.0,199011,284093,76034,1990-11-30
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103.0,199012,284093,76034,1990-12-31
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103.0,199010,284093,76034,1990-10-31
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156.0,199002,147300,76034,1990-02-28


In [12]:
# Create market routes (airport & city)
flights['market_air'] = flights['Origin'] + ' - ' + flights['Destination']
flights['market_city'] = flights['Origin City'] + ' - ' + flights['Destination City']
flights.head()

,Origin,Destination,Origin City,Destination City,Passengers,Seats,Flights,Distance,Fly Date,Origin Population,Destination Population,year-month,market_air,market_city
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254.0,200810,122049,86219,2008-10-31,MHK - AMW,"Manhattan, KS - Ames, IA"
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103.0,199011,284093,76034,1990-11-30,EUG - RDM,"Eugene, OR - Bend, OR"
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103.0,199012,284093,76034,1990-12-31,EUG - RDM,"Eugene, OR - Bend, OR"
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103.0,199010,284093,76034,1990-10-31,EUG - RDM,"Eugene, OR - Bend, OR"
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156.0,199002,147300,76034,1990-02-28,MFR - RDM,"Medford, OR - Bend, OR"


In [13]:
flights = flights.set_index('year-month').drop(columns=['Fly Date']).sort_index()
flights.head()

,Origin,Destination,Origin City,Destination City,Passengers,Seats,Flights,Distance,Origin Population,Destination Population,market_air,market_city
year-month,,,,,,,,,,,,
1990-01-31,SEA,ORD,"Seattle, WA","Chicago, IL",1713,4410,30,1721.0,5154164,16395048,SEA - ORD,"Seattle, WA - Chicago, IL"
1990-01-31,CLE,EWR,"Cleveland, OH","Newark, NJ",1476,4619,31,404.0,2103367,16868983,CLE - EWR,"Cleveland, OH - Newark, NJ"
1990-01-31,CRW,ROA,"Charleston, WV","Roanoke, VA",388,2100,21,114.0,307480,269195,CRW - ROA,"Charleston, WV - Roanoke, VA"
1990-01-31,CLE,EWR,"Cleveland, OH","Newark, NJ",1337,3348,31,404.0,2103367,16868983,CLE - EWR,"Cleveland, OH - Newark, NJ"
1990-01-31,CLE,EWR,"Cleveland, OH","Newark, NJ",2787,4888,52,404.0,2103367,16868983,CLE - EWR,"Cleveland, OH - Newark, NJ"


In [8]:
# # Add Year & Quarter Column for later data merge
# flights['year'] = flights['year-month'].dt.year
# flights['quarter'] = flights['year-month'].dt.quarter
# flights['month'] = flights['year-month'].dt.month

In [16]:
# Aggregate the dataframe so that each route has one row per year-month
flights = flights.groupby([pd.Grouper(freq='M'), 'market_city']).agg({'Passengers' : 'sum', 'Seats' : 'sum', 'Flights' : 'sum', 
                                                                      'Distance' : 'mean', 'Origin Population' : 'mean', 
                                                                      'Destination Population' : 'mean'}).reset_index().set_index('year-month')
flights.head()

,market_city,Passengers,Seats,Flights,Distance,Origin Population,Destination Population
year-month,,,,,,,
1990-01-31,"Abilene, TX - Dallas, TX",741,1018,7,158.0,147700,8019250
1990-01-31,"Akron, OH - Atlanta, GA",3742,5610,56,528.0,658558,3087755
1990-01-31,"Akron, OH - Birmingham, AL",75,99,1,585.0,658558,958585
1990-01-31,"Akron, OH - Chicago, IL",7863,20688,170,344.0,658558,16395048
1990-01-31,"Akron, OH - Cleveland, OH",0,123,1,40.0,658558,2103367


In [19]:
flights.to_csv('./data/clean/flightdata_city_bymonth.csv')